# ERC-4626: vault ecosystem comparison across chains

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [31]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [32]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 7014 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [33]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")
df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

display(df.head())

DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Protocol, Denomination, NAV, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain    Address                                                               
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080            vMAIA-bHERMES   
         0x00003b020004328e005a0011b99a00c100cb9040            vMAIA-bHERMES   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff  wrappedConvexCrvusdUsdt   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776                    stUSD   
         0x004626a008b1acdc4c74ab51644093b155e59a23                    stEUR   

                                                                                                  Name  \
Chain    Address                                                                                         
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080  Vote Maia - Burned Hermes: Aggregated Gov + Yi...   
         0x00003b020004328e005a0011b99a00c100cb9040  Vote Maia - Burned Hermes: Aggregated Gov + Yi...   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff                         Wrapped Convex crvUSD/USDT   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776                                        Staked USDA   
         0x004626a008b1acdc4c74ab51644093b155e59a23                                        Staked EURA   

                                                           Protocol  \
Chain    Address                                                      
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080  <generic 4626>   
         0x00003b020004328e005a0011b99a00c100cb9040  <generic 4626>   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff  <generic 4626>   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776  <generic 4626>   
         0x004626a008b1acdc4c74ab51644093b155e59a23  <generic 4626>   

                                                    Denomination        NAV  \
Chain    Address                                                              
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080         MAIA  83,522.99   
         0x00003b020004328e005a0011b99a00c100cb9040         MAIA  89,756.42   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff   cvxcrvUSDT       0.00   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776         USDA 913,365.47   
         0x004626a008b1acdc4c74ab51644093b155e59a23         EURA 310,316.25   

                                                      Mgmt fee   Perf fee  \
Chain    Address                                                            
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080  <unknown>  <unknown>   
         0x00003b020004328e005a0011b99a00c100cb9040  <unknown>  <unknown>   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff  <unknown>  <unknown>   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776  <unknown>  <unknown>   
         0x004626a008b1acdc4c74ab51644093b155e59a23  <unknown>  <unknown>   

                                                                        Shares  \
Chain    Address                                                                 
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080   83522.991343137221650124   
         0x00003b020004328e005a0011b99a00c100cb9040   89756.422928390178159298   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff                5.00187E-13   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776  820193.460116856888442176   
         0x004626a008b1acdc4c74ab51644093b155e59a23  288833.982019449690681523   

                                                             First seen  \
Chain    Address                                                          
Arbitrum 0x000000f0c01c6200354f240000b7003668b4d080 2024-10-17 02:54:55   
         0x00003b020004328e005a0011b99a00c100cb9040 2024-08-15 12:10:17   
         0x0021f89457a5dd4f709c68a2baa2ca94a4d2acff 2024-09-17 13:21:44   
         0x0022228a2cc5e7ef0274a7baa600d44da5ab5776 2024-01-11 08:32:25   
         0x004626a008b1acdc4c74ab51644093b155e59a23 2023-09-01 10:47:47   

                                                                         

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [34]:
nav_threshold = 10_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size(),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size(),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size(),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size(),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size(),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size(),
}).fillna(0).astype(int)

print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 10000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,
Arbitrum,1915,240,658,120,18,26,219
Avalanche,259,8,57,2,1,1,48
Base,1153,479,420,74,24,12,93
Berachain,230,1,17,14,0,0,2
Binance,316,9,84,11,0,0,25
Ethereum,2115,239,701,240,16,14,177
Hyperliquid,12,0,6,1,1,0,1
Mantle,37,0,11,0,0,0,5
Mode,56,0,26,2,0,0,0


# Vault deployment history

- Show how much history we have for each chain


In [35]:
# Assuming your DataFrame is named 'df'
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-04-03 20:38:18
Avalanche,2022-04-11 00:05:24,2025-04-07 12:03:28
Base,2023-08-04 15:36:07,2025-04-08 10:33:43
Berachain,2025-01-26 00:06:03,2025-04-08 17:27:44
Binance,2022-05-27 17:25:18,2025-04-08 11:29:07
Ethereum,2019-06-11 06:17:19,2025-04-07 10:13:47
Hyperliquid,2025-02-19 16:29:00,2025-04-07 15:02:00
Mantle,2023-08-22 13:25:16,2024-11-29 08:55:34
Mode,2024-03-06 23:58:43,2025-01-03 19:23:51


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [36]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"3,037,173,933.35",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,27690,<unknown>,<unknown>
Staked USDe,"2,180,494,599.88",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,69128,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"913,834,025.77",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,48138,<unknown>,<unknown>
Savings Dai,"516,290,716.10",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,72384,<unknown>,<unknown>
Bridged USDC (Stargate)Vault,"341,601,944.40",Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,128300,<unknown>,<unknown>
Usual Boosted USDC,"217,812,829.88",Ethereum,0xd63070114470f685b75b74d60eec7c1113d33a3d,USDC,17363,0.00,0.00
Fluid USD Coin,"216,518,662.58",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,17475,<unknown>,<unknown>
Staked USDX,"199,779,134.12",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5423,<unknown>,<unknown>
Fluid Tether USD,"196,596,047.45",Ethereum,0x5c20b550819128074fd538edf79791733ccedd18,USDT,10343,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [37]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain     Name                                            
Arbitrum  Fluid USD Coin                  28,848,237.23   
Avalanche HiYield Treasury Bill Vault      2,274,986.21   
Base      Spark USDC Vault                65,603,467.07   
          Spark USDC Vault                 6,337,727.64   
Berachain Bridged USDC (Stargate)Vault   341,601,944.40   
Binance   Staked USDX                    199,779,134.12   
Ethereum  Savings USDS                 3,037,173,933.35   
Mode      USDC Ironclad Vault                337,647.27   
Polygon   Compound USDC                   24,099,469.86   

                                                                           Address  \
Chain     Name                                                                       
Arbitrum  Fluid USD Coin                0x1a996cb54bb95462040408c06122d45d6cdb6096   
Avalanche HiYield Treasury Bill Vault   0x8475509d391e6ee5a8b7133221ce17019d307b3e   
Base      Spark USDC Vault              0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
          Spark USDC Vault              0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain Bridged USDC (Stargate)Vault  0x90bc07408f5b5eac4de38af76ea6069e1fcee363   
Binance   Staked USDX                   0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Ethereum  Savings USDS                  0xa3931d71877c0e7a3148cb7eb4463524fec27fbd   
Mode      USDC Ironclad Vault           0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Polygon   Compound USDC                 0x781fb7f6d845e3be129289833b04d43aa8558c42   

                                       Denomination  Total events   Mgmt fee  \
Chain     Name                                                                 
Arbitrum  Fluid USD Coin                       USDC         33516  <unknown>   
Avalanche HiYield Treasury Bill Vault          USDC           177  <unknown>   
Base      Spark USDC Vault                     USDC          5705       0.00   
          Spark USDC Vault                     USDC         31183  <unknown>   
Berachain Bridged USDC (Stargate)Vault       USDC.e        128300  <unknown>   
Binance   Staked USDX                          USDX          5423  <unknown>   
Ethereum  Savings USDS                         USDS         27690  <unknown>   
Mode      USDC Ironclad Vault                  USDC           183  <unknown>   
Polygon   Compound USDC                        USDC          4090       0.00   

                                         Perf fee  
Chain     Name                                     
Arbitrum  Fluid USD Coin                <unknown>  
Avalanche HiYield Treasury Bill Vault   <unknown>  
Base      Spark USDC Vault                   0.00  
          Spark USDC Vault              <unknown>  
Berachain Bridged USDC (Stargate)Vault  <unknown>  
Binance   Staked USDX                   <unknown>  
Ethereum  Savings USDS                  <unknown>  
Mode      USDC Ironclad Vault           <unknown>  
Polygon   Compound USDC                      0.00

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [38]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
USDC yVault-A,490167,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"571,093.38",530 days 05:16:22.945732,465206,24961
USDT yVault-A,458203,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"462,260.13",488 days 04:59:50.945732,407690,50513
pufETH,167818,Ethereum,0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"70,733.48",433 days 02:17:35.945732,151538,16280
Moonwell Flagship ETH,140433,Base,0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"14,423.55",300 days 09:04:55.945732,85346,55087
FARM_WETH,134041,Base,0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688,WETH,"1,095.24",579 days 21:35:07.945732,76065,57976
Bridged USDC (Stargate)Vault,128300,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"341,601,944.40",72 days 21:56:07.945732,80374,47926
Staked USDA,121750,Arbitrum,0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"913,365.47",453 days 13:29:45.945732,119736,2014
Moonwell Flagship USDC,92051,Base,0xc1256ae5ff1cf2719d4937adb3bbccab2e00a2ca,USDC,"27,156,258.18",300 days 09:04:55.945732,53178,38873
Beraborrow iBGT,86926,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"51,831.58",32 days 09:56:51.945732,47032,39894


## Most active vault per chain

- Display the number one vault per chain

In [39]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"121,750.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"913,365.47",453 days 13:29:45.945732,119736,2014
Avalanche,GoGoPool Liquid Staking Token,"66,463.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"832,304.08",734 days 00:51:25.945732,41197,25266
Base,Moonwell Flagship ETH,"140,433.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"14,423.55",300 days 09:04:55.945732,85346,55087
Berachain,Bridged USDC (Stargate)Vault,"128,300.00",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"341,601,944.40",72 days 21:56:07.945732,80374,47926
Binance,kUSDT,"42,614.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,712,184.69",666 days 14:53:27.945732,25239,17375
Ethereum,pufETH,"167,818.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"70,733.48",433 days 02:17:35.945732,151538,16280
Hyperliquid,wHYPE,534.00,0x2831775cb5e64b1d892853893858a261e898fbeb,WHYPE,"167,349.65",26 days 23:51:10.945732,411,123
Mantle,Karak - mETH,"13,024.00",0x8529019503c5bd707d8eb98c5c87bf5237f89135,mETH,490.09,350 days 03:50:04.945732,8038,4986
Mode,Renzo aggregator,"21,417.00",0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,WETH,209.76,397 days 22:03:27.945732,12187,9230


## Oldest vaults

- Show oldest vaults

In [40]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1047 days 06:47:04.945732,WETH,18.00,"2,335.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1091 days 05:04:48.945732,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,586 days 06:08:17.945732,WETH,0.97,"1,117.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,72 days 21:56:07.945732,USDC.e,"341,601,944.40","128,300.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1016 days 01:09:31.945732,NRCH,"1,188,908.73","1,195.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1079 days 10:36:33.945732,THOR,"79,360,216.48","19,570.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,350 days 03:50:04.945732,mETH,490.09,"13,024.00"
Mode,0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,Renzo aggregator,397 days 22:03:27.945732,WETH,209.76,"21,417.00"
Polygon,0x73958d46b7aa2bc94926d8a215fa560a5cdca3ea,Wrapped Aave Polygon GHST,1070 days 18:15:21.945732,aPolGHST,"1,144,866.24","14,510.00"
